In [2]:
import pandas as pd 
import pymysql 
import pymysql.cursors
import numpy as np
from scipy.stats import chisquare
from scipy import stats
import scipy as sp
import statsmodels.api as sm
from statsmodels.formula.api import ols, logit, glm
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

In [3]:
# dog1 = pd.read_excel("c:\import\gihosung\dog1_test_result.xlsx")
# dog2 = pd.read_excel("c:\import\gihosung\dog2_test_result.xlsx")
# cat_zero = pd.read_excel("c:\import\gihosung\cat_zero_test_result.xlsx")
# cat_zero2 = pd.read_excel("c:\import\gihosung\cat_zero2_test_result.xlsx")
cat_dongdong = pd.read_excel("c:\import\gihosung\s_1_test_result\cat_dongdong_test_result.xlsx")

In [45]:
x_x = cat_dongdong[['p_cow' , 'p_chicken' , 'p_fish' , 'p_turkey' , 'p_sheep' ,  'p_salmon' , 'p_tuna'  , 'o_chicken' , 'o_fish' , 'flavor']]
y_y = cat_dongdong[['little_score']]

In [46]:
# 변수선택을 통해 형성한 모델의 AIC를 구하는 함수
# AIC가 낮을 수록 모델이 좋다고 평가된다.

def processSubset(X,y,feature_set):
    model = sm.OLS(y,X[list(feature_set)]) # Modeling
    regr = model.fit() # model fitting
    AIC = regr.aic # model's AIC
    return {"model" : regr, "AIC" : AIC}
    
# 전체 변수의 AIC test
feature_columns = ['p_cow' , 'p_chicken' , 'p_fish' , 'p_turkey' , 'p_sheep' ,  'p_salmon' , 'p_tuna'  , 'o_chicken' , 'o_fish' , 'flavor']
processSubset(X=x, y=y, feature_set = feature_columns) 

{'model': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x17c3a1d84f0>,
 'AIC': 32.00086653053452}

In [47]:
x_x = sm.add_constant(x_x ,has_constant = "add")
x.head()

C:\visual studio\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


,const,p_cow,p_chicken,p_fish,p_turkey,p_sheep,p_salmon,p_tuna,o_chicken,o_fish,flavor
0,1.0,0,1,0,0,0,0,0,0,0,1
1,1.0,0,1,1,0,0,0,0,0,0,1
2,1.0,0,0,0,0,1,1,0,1,1,1
3,1.0,0,0,1,0,0,0,0,1,1,1
4,1.0,0,1,0,1,0,1,0,1,1,0


In [48]:
import time
import itertools 

# getBest : 가장 낮은 AIC를 가지는 모델을 선택하고 저장하는 함수 https://todayisbetterthanyesterday.tistory.com/9

def getBest(X,y,k):
    tic = time.time()      # 시작 시간
    results = []           # 결과 저장 공간
    for combo in itertools.combinations(X.columns.difference(['const']),k) :
		# 각 변수 조합을 고려한 경우의수
        
        combo = (list(combo)+['const'])
        # 상수항을 추가하여 combo를 결성
        
        results.append(processSubset(X,y,feature_set = combo)) # 모델링된것을 저장
        
        # 만약 k=2이면 여기서 두가지 변수만 뽑아서 경우의 수를 분석하여
        # 저장 후 그 중 AIC가 가장 낮은 모델을 선택하도록 함
	
    models = pd.DataFrame(results) # 데이터프레임으로 모델결과 변환
    best_model = models.loc[models['AIC'].argmin()] # argmin은 최소값의 인덱스를 뽑는 함수
    toc = time.time()       # 종료 시간
    print("Processed", models.shape[0], "models on", k, "predictors in",(toc - tic),"seconds.")
    
    return best_model
    
model_all = getBest(X=x, y = y, k=10)

model_all


Processed 1 models on 10 predictors in 0.0029904842376708984 seconds.


model    <statsmodels.regression.linear_model.Regressio...
AIC                                               33.98958
Name: 0, dtype: object

In [49]:
#전진 선택법(Step=1)
 
#변수 선택법
def processSubset(x,y, feature_set):
    model = sm.OLS(y,x[list(feature_set)])  #modeling
    regr = model.fit()  #모델학습
    AIC = regr.aic    #모델의 AIC
    return {"model" : regr , "AIC" : AIC}
 
#print(processSubset(x=trainX,y=trainY,feature_set=featureColums[0:5]))
 
def forward(x,y,predictors):
    remainingPredictors = [p for p in x.columns.difference(['const'])
                           if p not in predictors]
    tic=time.time()
    results=[]
    for p in remainingPredictors:
        results.append(processSubset(x=x,y=y,feature_set=predictors+[p]+['const']))
    #데이터프레임으로 변환
    models = pd.DataFrame(results)
 
    #AIC가 가장 낮은 것을 선택
    bestModel = models.loc[models['AIC'].argmin()] #index
    toc = time.time()
    print("Processed ", models.shape[0],"models on", len(predictors)+1,
          "predictors in",(toc-tic))
    print("Selected predictors:",bestModel['model'].model.exog_names,
          'AIC : ',bestModel[0])
    return bestModel
 
#전진선택법 모델
def forward_model(x,y):
    fModels = pd.DataFrame(columns=["AIC","model"])
    tic = time.time()
    #미리 정의된 데이터 변수
    predictors = []
    #변수1~10개 : 0~9 -> 1~10
    for i in range(1,len(x.columns.difference(['const']))+1):
        forwardResult= forward(x,y,predictors)
        if i > 1:
            if forwardResult['AIC'] > fmodelBefore:
                break
        fModels.loc[i] = forwardResult
        predictors = fModels.loc[i]["model"].model.exog_names
        fmodelBefore = fModels.loc[i]["AIC"]
        predictors = [k for k in predictors if k != 'const']
    toc = time.time()
    print("Total elapesed time : ", (toc - tic), "seconds.")
 
    return (fModels['model'][len(fModels['model'])])

predictors = x_x[[]]
 
forwordBestModel=forward_model(x_x,y_y)

 
print(forwordBestModel.summary())
 

 


Processed  10 models on 1 predictors in 0.008970260620117188
Selected predictors: ['p_turkey', 'const'] AIC :  <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x0000017C3A1D23D0>
Processed  9 models on 2 predictors in 0.008936405181884766
Selected predictors: ['p_turkey', 'p_chicken', 'const'] AIC :  <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x0000017C3A1B7370>
Total elapesed time :  0.02491474151611328 seconds.
                            OLS Regression Results                            
Dep. Variable:           little_score   R-squared:                       0.142
Model:                            OLS   Adj. R-squared:                  0.081
Method:                 Least Squares   F-statistic:                     2.315
Date:                Thu, 13 Jan 2022   Prob (F-statistic):              0.150
Time:                        16:18:09   Log-Likelihood:                -29.283
No. Observations:                  16   AIC:            

C:\visual studio\lib\site-packages\scipy\stats\stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [50]:
#후진제거법
 
def backward(x,y,predictors):
    tic = time.time()
    results=[]
    #데이터 변수들이 미리정의된 predictors 조합확인
    for combo in itertools.combinations(predictors, len(predictors)-1):
        results.append(processSubset(x,y,list(combo)+['const']))
    models = pd.DataFrame(results)
    #가장 낮은 AIC를 가진 모델을 선택
    bestModel = models.loc[models['AIC'].argmin()]
    toc = time.time()
    print("Processed",models.shape[0],"models on",len(predictors)-1,
          "predictors in",(toc - tic))
    print("Selected predictors :",bestModel['model'].model.exog_names,
          ' AIC:',bestModel[0])
    return bestModel
 
def backword_model(x,y):
    BModels = pd.DataFrame(columns=["AIC","model"])
    tic = time.time()
    #미리 정의된 데이터 변수
    predictors = x.columns.difference(['const'])
    BmodelBefore = processSubset(x,y,predictors)['AIC']
    while(len(predictors)>1):
        backwardResult=backward(trainX,trainY,predictors)
        if backwardResult['AIC'] > BmodelBefore:
            break
        BModels.loc[len(predictors)-1] = backwardResult
        predictors = BModels.loc[len(predictors)-1]["model"].model.exog_names
        BmodelBefore = backwardResult["AIC"]
        predictors = [ k for k in predictors if k != 'const']
 
    toc = time.time()
    print("Total elapsed time :",(toc - tic), "seconds.")
    return (BModels["model"].dropna().iloc[0])
 
#backwardBestModel = backword_model(trainX,trainY)

In [51]:
def Stepwise_model(x,y):
    stepModels = pd.DataFrame(columns=["AIC","model"])
    tic = time.time()
    predictors = []
    SmodelBefore = processSubset(x,y,predictors+['const'])['AIC']
    #변수 1~10개 : 0~9 -> 1~10
    for i in range(1, len(x.columns.difference(['const']))+1):
        forwardResult = forward(x,y,predictors)
        print("forward")
        stepModels.loc[i] = forwardResult
        predictors = stepModels.loc[i]["model"].model.exog_names
        predictors = [k for k in predictors if k != 'const']
        backwordResult = backward(x,y,predictors)
        if backwordResult['AIC'] < forwardResult['AIC']:
            stepModels.loc[i] = backwordResult
            predictors=stepModels.loc[i]["model"].model.exog_names
            smodelBefore=stepModels.loc[i]["AIC"]
            predictors=[k for k in predictors if k != 'const']
            print('backward')
        if stepModels.loc[i]["AIC"] > SmodelBefore:
            break
        else:
            smodelBefore = stepModels.loc[i]["AIC"]
    toc=time.time()
    print("Total elapsed time : ", (toc - tic), "seconds")
    return (stepModels['model'][len(stepModels['model'])])
 
stepwiseBestModel = Stepwise_model(x_x,y_y)

print(stepwiseBestModel.summary())


Processed  10 models on 1 predictors in 0.010943174362182617
Selected predictors: ['p_turkey', 'const'] AIC :  <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x0000017C3A044430>
forward
Processed 1 models on 0 predictors in 0.0009949207305908203
Selected predictors : ['const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x0000017C3A1D2880>
Processed  9 models on 2 predictors in 0.014461517333984375
Selected predictors: ['p_turkey', 'p_chicken', 'const'] AIC :  <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x0000017C3A2C83D0>
forward
Processed 2 models on 1 predictors in 0.001996755599975586
Selected predictors : ['p_turkey', 'const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x0000017C3A1AB640>
backward
Processed  9 models on 2 predictors in 0.010963678359985352
Selected predictors: ['p_turkey', 'p_chicken', 'const'] AIC :  <statsmodels.regression.linear_model.Regres

C:\visual studio\lib\site-packages\scipy\stats\stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [ ]:
#https://zephyrus1111.tistory.com/65

## 전진 단계별 선택법

